In [ ]:
# bioinfo1 기말발표 YoA _ 한예원 2024-21904 

# Workflow 

featureCount_res_raw.tsv  # Guided Mission 1 의 cnt.tsv

TPM 계산, Log(CLIP enrichment), Log(RdenChange) 계산  (done)

여기서 0 하나도 없는 열만 추출 (done)

# 단, control values 다 남겨놓기 (통계처리에 필요))

id 리스트 뽑아서 GO term 대응 

BP만 뽑기

BP GO-slim(mouse)로 대응시키기 

# 통계처리 잘 생각하기 
이게 각 BP 당

하나의 BP = pathway (gene set)  __ 이 pathway가 유의미하게 증진되었는가? 

fisher or mHG 선택하기 (multiple test correction해야함)



In [3]:
#calculate TPM 

import pandas as pd

# 1. featureCounts 결과 불러오기
df = pd.read_csv("/fast/yewon/colab-biolab/featureCount_res_raw.tsv", sep="\t", comment='#')

# 2. 유전자 길이 (bp → kb)
df['Length_kb'] = df['Length'] / 1000

# 3. count 열만 선택 (샘플 열) — 보통 7열부터 끝까지가 count
count_cols = df.columns[6:-1]  # 'Length_kb' 앞까지만

# 4. RPK 계산
rpk = df[count_cols].div(df['Length_kb'], axis=0)

# 5. 샘플별 scaling factor 계산 (RPK 합 / 1e6)
scaling_factors = rpk.sum(axis=0) / 1e6

# 6. TPM 계산
tpm = rpk.div(scaling_factors, axis=1)

# 7. Gene ID 열 추가하고 저장
tpm.insert(0, 'Geneid', df['Geneid'])
tpm.to_csv("/fast/yewon/colab-biolab/featureCount_res_w_TPM.tsv", sep='\t', index=False)

print("✅ TPM 계산 완료! 'featureCount_res_w_TPM.tsvv' 파일이 생성되었습니다.")


✅ TPM 계산 완료! 'featureCount_res_w_TPM.tsvv' 파일이 생성되었습니다.


In [4]:
# 기존 df는 tpm 계산된 상태
df = tpm.copy()

# 1. 새로운 지표 계산
df['clip_enrichment'] = df['CLIP-35L33G.bam'] / df['RNA-control.bam']
df['rden_change'] = (df['RPF-siLin28a.bam'] / df['RNA-siLin28a.bam']) / \
                    (df['RPF-siLuc.bam'] / df['RNA-siLuc.bam'])

# 2. 필요한 열 중 하나라도 0이 있으면 제거
required_cols = [
    'CLIP-35L33G.bam', 'RNA-control.bam',
    'RPF-siLin28a.bam', 'RNA-siLin28a.bam',
    'RPF-siLuc.bam', 'RNA-siLuc.bam'
]

df_filtered = df[(df[required_cols] != 0).all(axis=1)]

# 3. 저장
df_filtered.to_csv("featureCount_res_TPM_ClipEnrich_RdenChange.tsv", sep='\t', index=False)

print("✅ 저장 완료: featureCount_res_TPM_ClipEnrich_RdenChange.tsv")


✅ 저장 완료: featureCount_res_TPM_ClipEnrich_RdenChange.tsv


In [5]:
df_filtered.head()

,Geneid,CLIP-35L33G.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,clip_enrichment,rden_change
15,ENSMUSG00000102343.2,9.756596,11.878639,42.787984,40.466618,0.116285,0.208447,0.821356,0.527600
19,ENSMUSG00000025902.14,0.249740,0.109526,0.472667,0.368367,0.066477,0.089372,2.280183,0.579688
21,ENSMUSG00000102269.2,0.531265,12.756331,2.639414,1.292968,0.106060,0.427766,0.041647,0.121458
26,ENSMUSG00000098104.2,3.918488,22.399719,4.986799,3.587447,0.215801,0.676957,0.174935,0.229327
30,ENSMUSG00000103922.2,34.002534,46.447811,24.528506,32.887731,6.231771,9.973871,0.732059,0.837743


In [5]:
# featureCount_res_TPM_ClipEnrich_RdenChange.tsv

# ENS EMBL ID - to GO term 

data_df = pd.read_csv("featureCount_res_TPM_ClipEnrich_RdenChange.tsv", sep="\t", comment='#')

In [6]:
len(data_df)

16201

In [ ]:
# GO term mapping 

In [8]:
!pip install gseapy networkx

In [9]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import gseapy as gp #1
import networkx as nx

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
data = pd.read_csv('featureCount_res_TPM_ClipEnrich_RdenChange.tsv', sep='\t', index_col=0) #2
data.head()

,CLIP-35L33G.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,clip_enrichment,rden_change
Geneid,,,,,,,,
ENSMUSG00000102343.2,9.756596,11.878639,42.787984,40.466618,0.116285,0.208447,0.821356,0.527600
ENSMUSG00000025902.14,0.249740,0.109526,0.472667,0.368367,0.066477,0.089372,2.280183,0.579688
ENSMUSG00000102269.2,0.531265,12.756331,2.639414,1.292968,0.106060,0.427766,0.041647,0.121458
ENSMUSG00000098104.2,3.918488,22.399719,4.986799,3.587447,0.215801,0.676957,0.174935,0.229327
ENSMUSG00000103922.2,34.002534,46.447811,24.528506,32.887731,6.231771,9.973871,0.732059,0.837743


In [32]:
# #  수업중 실습 코드  : GO term 대응 위해 biomart 사용하는 부분만 사용 
# data = data.index.to_series().str.split('.').str.get(0).tolist() #6
# data[:5]
 

['ENSMUSG00000102343',
 'ENSMUSG00000025902',
 'ENSMUSG00000102269',
 'ENSMUSG00000098104',
 'ENSMUSG00000103922']

In [34]:
import pandas as pd

# starting point: your DataFrame ‘data’, whose index are "ENSMUSG000000…"
# strip off the version suffix and build a new Series in one go:
ids = data.index.to_series().str.split('.').str[0]

# convert to a one-column DataFrame (gives you column header too):
df_ids = ids.to_frame(name='ensembl_gene_id')

# write that out, no need for "col_index":
df_ids.to_csv("ALL-16K-Ids.tsv", sep="\t", index=False)


In [23]:
len(data)

16201

In [42]:
# # 수업중 실습 코드(13주차) 일부 활용: GO term 대응 위해 gseapy.biomart 활용하는 파트 변형하여 사용 


data = pd.read_csv('ALL-16K-Ids.tsv', sep='\t', index_col=0) #2

In [52]:
import pandas as pd
import gseapy as gp

# ─── Helper: simple batching ────────────────────────────────────────────────────
def batched(iterable, n):
    items = []
    for item in iterable:
        items.append(item)
        if len(items) == n:
            yield items
            items = []
    if items:
        yield items

# ─── Load your 16 K IDs robustly ───────────────────────────────────────────────
def load_ensembl_ids(path):
    # Try reading with no index_col first
    df = pd.read_csv(path, sep='\t', header=None)
    # Assume the first column is your Ensembl IDs
    ids = df.iloc[:, 0].astype(str).tolist()
    print(f"Loaded {len(ids)} IDs (first 5):", ids[:5])
    return ids

ensembl_ids = load_ensembl_ids('ALL-16K-Ids.tsv')

# ─── Set up Biomart ────────────────────────────────────────────────────────────
bm = gp.biomart.Biomart()  
DATASET = 'mmusculus_gene_ensembl'

# ─── Query in batches ──────────────────────────────────────────────────────────
results = []
BATCH_SIZE = 200

for batch in batched(ensembl_ids, BATCH_SIZE):
    queries = {'ensembl_gene_id': batch}
    df_batch = bm.query(
        dataset=DATASET,
        attributes=[
            'ensembl_gene_id',     # input ID
            'external_gene_name',  # mouse symbol
            'go_id',                # GO term ID
            # 'go_domain'                # GO domain 
            'namespace_1003'       # GO domain/namespace
        ],
        filters=queries
    )
    if not df_batch.empty:
        results.append(df_batch)

# ─── Combine & clean ───────────────────────────────────────────────────────────
if not results:
    raise RuntimeError("No GO annotations retrieved. Check your IDs or the dataset/filter names.")

df_all = pd.concat(results, ignore_index=True)
df_all = df_all.dropna(subset=['go_id'])  # drop genes with no GO

# ─── Inspect & save ───────────────────────────────────────────────────────────
print("Total rows (gene–GO pairs):", len(df_all))
print(df_all.head())

df_all.to_csv('mouse_16K_GO_mappings_final.tsv', sep='\t', index=False)


Loaded 16202 IDs (first 5): ['ensembl_gene_id', 'ENSMUSG00000102343', 'ENSMUSG00000025902', 'ENSMUSG00000102269', 'ENSMUSG00000098104']
Total rows (gene–GO pairs): 218244
      ensembl_gene_id external_gene_name       go_id      namespace_1003
0  ENSMUSG00000001138              Cnnm3  GO:0010960  biological_process
1  ENSMUSG00000001138              Cnnm3  GO:0005886  cellular_component
2  ENSMUSG00000001138              Cnnm3  GO:0006811  biological_process
4  ENSMUSG00000001143             Lman2l  GO:0016020  cellular_component
5  ENSMUSG00000001143             Lman2l  GO:0030246  molecular_function


In [ ]:
# biomart 결과 csv 

# mouse_16K_GO_mappings.tsv
mouse_16K_GO_mappings_final.tsv

grep "biological_process" mouse_16K_GO_mappings_final.tsv > mouse_16K_GO_mappings_BP.tsv  # 95846

In [ ]:
# 이제 BP-slim 대응시키기 

In [ ]:
#
awk -F'\t' -v OFS='\t' '
  BEGIN {
    # GAF 2.2 header
    print "!gaf-version: 2.2"
  }
  {
    # 1: DB          -> hyphen
    # 2: DB Object ID -> $1
    # 3: DB Object Symbol         -> hyphen
    # 4: Qualifier                -> hyphen
    # 5: GO ID                     -> $2
    # 6: DB:Reference             -> hyphen
    # 7: Evidence Code            -> hyphen
    # 8: With/From                -> hyphen
    # 9: Aspect                   -> BP
    # 10: DB Object Name          -> hyphen
    # 11: DB Object Synonym       -> hyphen
    # 12: DB Object Type          -> hyphen
    # 13: Taxon                   -> hyphen
    # 14: Date                    -> hyphen
    # 15: Assigned By             -> hyphen
    # 16: Annotation Extension    -> hyphen
    # 17: Gene Product Form ID    -> hyphen
    print "-", 
          $1,
          "-", 
          "-", 
          $3,
          "-", 
          "-", 
          "-", 
          "BP", 
          "-", 
          "-", 
          "-", 
          "taxon:10090", 
          "-", 
          "-", 
          "-", 
          "-"
  }
' mouse_16K_GO_mappings_BP.tsv > mouse_16K_GO_mappings_BP.gaf


In [ ]:
# install owltools

In [ ]:
# wget https://current.geneontology.org/ontology/subsets/goslim_mouse.obo



cd owltools/

awk '
  BEGIN { inTerm = 0; id = "" }
  /^\[Term\]/   { inTerm = 1; id = ""; next }
  /^$/          { inTerm = 0; next }
  inTerm && /^id: GO/ {
    id = $2
  }
  inTerm && /^namespace: biological_process/ {
    if (id) print id
    id = ""
  }
' goslim_mouse.obo > bp_slim_ids.txt

/fast/yewon/colab-biolab/owltools/bp_slim_ids.txt

In [ ]:
# map GO slim (BP terms)

# didin't use this
/fast/yewon/colab-biolab/owltools/owltools /fast/yewon/colab-biolab/owltools/go.obo --gaf /fast/yewon/colab-biolab/mouse_16K_GO_mappings_BP.gaf --map2slim --idfile /fast/yewon/colab-biolab/owltools/bp_slim_ids.txt --write-gaf /fast/yewon/colab-biolab/mouse_BP-GOslim.gaf

#result 
/fast/yewon/colab-biolab/mouse_BP-GOslim.gaf

# Number of annotations rewritten: 2361


In [ ]:
 
awk -F'\t' -v OFS='\t' '{ sub(/^-:/,"",$2); print }'  /fast/yewon/colab-biolab/mouse_BP-GOslim.gaf > temp.tsv
mv temp.tsv /fast/yewon/colab-biolab/mouse_BP-GOslim.gaf
 
# cd owltools
f1="/fast/yewon/colab-biolab/mouse_BP-GOslim.gaf"
f2="/fast/yewon/colab-biolab/mouse_16K_GO_mappings_BP.gaf"  # original input

awk -F"\t" -v OFS="\t" 'NR==FNR {data[$2]=$5;next;}($2 in data){print $2, $5, data[$2]}' $f1 $f2 > /fast/yewon/colab-biolab/mouse_ALL_BP-ID_GO_GoSlim.tsv  # 93242 


In [ ]:
"/fast/yewon/colab-biolab/mouse_ALL_BP-ID_GO_GoSlim.tsv" # 10812 엔트리 


# 여기에서 BP term 만 골라내기  


# awk '{print $1"\t"$3}' $myf | sort | uniq > temp-ID_GOslim.tsv

# awk '{print $1}' temp-ID_GOslim.tsv | sort | uniq | wc -l 
# # 10812


bp="/fast/yewon/colab-biolab/owltools/bp_slim_ids.txt"  # 이게 문제인거같아ㅏㅏ 
myf="/fast/yewon/colab-biolab/mouse_ALL_BP-ID_GO_GoSlim.tsv"
awk -F"\t" -v OFS="\t" 'NR==FNR{data[$1];next;} ($3 in data)' $bp $myf > /fast/yewon/colab-biolab/mouse_ALL_BP-ID_GO_GoSlim-BP.tsv  # 10317  # 거의 큰 차이 안나지만 이거로하기      

In [ ]:
awk -F"\t" -v OFS="\t" '{print $1"\t"$3}' mouse_ALL_BP-ID_GO_GoSlim-BP.tsv | sort | uniq > mouse_ALL_BP-ID_GoSlim-BP.tsv #10780


awk -F"\t" -v OFS="\t" 'NR==FNR{data[$2]=$1;next} ()'
featureCount_with_log2.tsv 

In [ ]:
# concat 
awk -F'\t' '
  BEGIN { OFS=FS }
  {
    gene = $1
    term = $2         # <-- use the GoSlim column here
    if (term != "") {
      key = gene "|" term
      if (!seen[key]++) {
        if (out[gene] == "")
          out[gene] = term
        else
          out[gene] = out[gene] ";" term
      }
    }
  }
  END {
    for (g in out)
      print g, out[g]
  }
' /fast/yewon/colab-biolab/mouse_ALL_BP-ID_GoSlim-BP.tsv  \
  > /fast/yewon/colab-biolab/mouse_ALL_BP-ID_GoSlim-BP-concat.tsv


In [1]:
data = pd.read_csv('featureCount_res_TPM_ClipEnrich_RdenChange.tsv', sep='\t', index_col=0) 
# Geneid  CLIP-35L33G.bam RNA-control.bam RNA-siLin28a.bam        RNA-siLuc.bam   RPF-siLin28a.bam        RPF-siLuc.bam   clip_enrichment rden_change

# output should be : 
# Geneid  CLIP-35L33G.bam RNA-control.bam RNA-siLin28a.bam        RNA-siLuc.bam   RPF-siLin28a.bam        RPF-siLuc.bam   clip_enrichment rden_change   log2_clip_enrichment        log2_rden_change

import pandas as pd
import numpy as np

# Add log2-transformed columns
data['log2_clip_enrichment'] = np.log2(data['clip_enrichment'].replace(0, np.nan))
data['log2_rden_change'] = np.log2(data['rden_change'].replace(0, np.nan))

# (Optional) Save to file
data.to_csv('featureCount_with_log2.tsv', sep='\t')


NameError: name 'pd' is not defined

In [ ]:
awk -F"\t" -v OFS="\t" '{ split($1, a, "."); $1 = a[1]; print }' featureCount_with_log2.tsv > temp.tsv
mv temp.tsv featureCount_with_log2.tsv

In [28]:
# Define input files
go_terms_file="/fast/yewon/colab-biolab/mouse_ALL_BP-ID_GoSlim-BP-concat.tsv"
data_file="featureCount_with_log2.tsv"
output_file="mouse_ALL_BP-ID_allvals_logClip_logRdenFC_GOconcat.tsv"  #

# Extract relevant columns by joining on gene ID
awk -F"\t" -v OFS="\t" '
  NR==FNR {
    if ($2 != "") go_func[$1] = $2
    next
  }
  ($1 in go_func) {
    print $0, go_func[$1]
  }
' "$go_terms_file" "$data_file" > "$output_file" # 10317
# head $output_filec


In [55]:
# Define input files
go_terms_file="/fast/yewon/colab-biolab/mouse_ALL_BP-ID_GoSlim-BP-concat.tsv"
data_file="featureCount_with_log2.tsv"
output_file="mouse_ALL_BP-ID_logClip_logRdenFC_GOconcat.tsv"  # all_integral_membrane_mf-ID_logClip_logRdenFC_GOconcat.tsv" 

# Extract relevant columns by joining on gene ID
awk -F"\t" -v OFS="\t" '
  NR==FNR {
    if ($2 != "") go_func[$1] = $2
    next
  }
  ($1 in go_func) {
    print $1, $8, $9, go_func[$1]
  }
' "$go_terms_file" "$data_file" > "$output_file" # 10317



SyntaxError: unterminated string literal (detected at line 7) (3339049799.py, line 7)

In [ ]:
awk -F"\t" '{
  n = split($4, terms, ";")
  for (i = 1; i <= n; i++) {
    print $1 "\t" $2 "\t" $3 "\t" terms[i]
  }
}' mouse_ALL_BP-ID_logClip_logRdenFC_GOconcat.tsv \
> mouse_ALL_BP-ID_logClip_logRdenFC_GOconcat_exploded.tsv


In [ ]:
df = pd.read_csv(
    'mouse_ALL_BP-ID_logClip_logRdenFC_GOconcat_exploded.tsv',
    sep='\t', header=None,
    names=['gene','scoreA','scoreB','goslim']
)


: 

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt

# ─── Load Preprocessed Data ───────────────────────────────────────────────────
df = pd.read_csv(
    'mouse_ALL_BP-ID_logClip_logRdenFC_GOconcat_exploded.tsv',
    sep='\t', header=None,
    names=['gene', 'scoreA', 'scoreB', 'goslim']
)

# ─── Group by GO term ─────────────────────────────────────────────────────────
grouped = (
    df.groupby('goslim')
      .agg(
         gene_count=('gene', 'nunique'),
         scoreA_mean=('scoreA', 'mean'),
         scoreB_mean=('scoreB', 'mean')
      )
      .reset_index()
)

# ─── Mann–Whitney U test (scoreA in vs out per GO term) ───────────────────────
gene_to_scoreA = df.set_index('gene')['scoreA'].to_dict()
all_genes = set(gene_to_scoreA)

pvals = []
for term in grouped['goslim']:
    genes_in = set(df[df['goslim'] == term]['gene'])
    genes_out = all_genes - genes_in

    s_in = [gene_to_scoreA[g] for g in genes_in if g in gene_to_scoreA]
    s_out = [gene_to_scoreA[g] for g in genes_out if g in gene_to_scoreA]

    if len(s_in) >= 3 and len(s_out) >= 3:
        _, p = mannwhitneyu(s_in, s_out, alternative='two-sided')
    else:
        p = np.nan
    pvals.append(p)

grouped['pA']   = pvals
grouped['fdrA'] = multipletests(grouped['pA'].fillna(1), method='fdr_bh')[1]
grouped['sigA'] = -np.log10(grouped['fdrA'].replace(0, np.nan))

# ─── Plotting Bubble Chart ────────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(8, 6))
scatter = ax.scatter(
    grouped['scoreA_mean'],
    grouped['scoreB_mean'],
    s=grouped['gene_count'] * 3,
    c=grouped['sigA'],
    cmap='YlOrBr',
    edgecolor='black'
)

# Add grid lines every 0.25
x_min, x_max = grouped['scoreA_mean'].min(), grouped['scoreA_mean'].max()
y_min, y_max = grouped['scoreB_mean'].min(), grouped['scoreB_mean'].max()
ax.set_xticks(np.arange(np.floor(x_min), np.ceil(x_max) + 0.25, 0.25))
ax.set_yticks(np.arange(np.floor(y_min), np.ceil(y_max) + 0.25, 0.25))
ax.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)

# Reference lines at 0
ax.axhline(0, color='gray', linestyle='--', linewidth=0.8)
ax.axvline(0, color='gray', linestyle='--', linewidth=0.8)

# Colorbar
cbar = fig.colorbar(scatter, ax=ax, pad=0.02)
cbar.set_label(r'$-\log_{10}(\mathrm{FDR})$', rotation=270, labelpad=15)

# Axis labels and title
ax.set_xlabel(r'Mean $\log_{2}$(Clip Enrichment)', fontsize=11)
ax.set_ylabel(r'Mean $\log_{2}$(Rden Fold Change)', fontsize=11)
ax.set_title('GO-Slim Term Enrichment\n(size = # genes, color = term confidence)', fontsize=12)

plt.tight_layout()
plt.savefig('goslim_bubble_plot.svg')
plt.show()
